In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import random
import math
import pickle
import matplotlib.pyplot as plt
import numpy as np
import copy

c:\Users\Andrew\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
embedding = nn.Embedding(num_embeddings=3000, embedding_dim=32)

In [16]:
def test(a:int = 2):
    print(a)

test()

2


In [42]:
import models

In [45]:

class missingWordPredictor(nn.Module):
    def __init__(
        self,
        num_embeddings : int,
        embedding_dim : int,
        
        phrase_len : int = 7
    ):
        
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim
        self.phrase_len = phrase_len
        super(missingWordPredictor, self).__init__()

        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim)

        self.linear1 = nn.Linear(in_features=(phrase_len-1)*embedding_dim, out_features=100)
        self.linear2 = nn.Linear(in_features=100, out_features=100)
        self.linear3 = nn.Linear(in_features=100, out_features=embedding_dim)

    def forward(self, 
    X : torch.LongTensor #LongTensor of lookup table indices corresponding to the 7 words in the phrase.
    ):
        X = torch.cat([X[:,:int((self.phrase_len-1)/2)], X[:,int((self.phrase_len+1)/2):]], dim=1) #Remove the middle item



        X = self.embedding(X) #Convert indices into embedding tensors

        print(X.shape)

        X = self.linear1(X)
        X = torch.relu(X)
        X = self.linear2(X)
        X = torch.relu(X)
        X = self.linear3(X)

        return X
        


model = missingWordPredictor(3000, 10)

In [7]:
t = torch.rand(2,4,4)

In [8]:
t

tensor([[[0.9264, 0.6551, 0.6626, 0.2545],
         [0.1405, 0.9667, 0.0270, 0.8462],
         [0.3715, 0.8983, 0.1324, 0.7176],
         [0.9019, 0.8360, 0.0359, 0.9725]],

        [[0.1654, 0.8290, 0.7815, 0.5353],
         [0.7525, 0.6947, 0.0240, 0.5668],
         [0.2977, 0.7322, 0.6660, 0.6194],
         [0.6403, 0.9290, 0.1911, 0.7290]]])

In [29]:
import string
text_bible = []
with open('data/bible.txt','r') as f:
    for line in f:
        for word in line.split():
            text_bible.append(word.translate(str.maketrans('','', string.punctuation)).lower())           

   


In [32]:
embedding = nn.Embedding(3000,10)

In [33]:
torch.save(embedding.state_dict(), "embedding_weights.pt")

In [39]:
embedding2 = nn.Embedding(3000,10)
embedding2.load_state_dict(torch.load("trained_embedding_weights.pt"))

<All keys matched successfully>

In [37]:
with open('bible_lookup_table.pkl', 'rb') as f:
    lt = pickle.load(f)

In [46]:
def get_embedding(word):
    return embedding2(torch.LongTensor([lt[word]]))

In [48]:
one = get_embedding('one')
two = get_embedding('two')
three = get_embedding('three')

In [49]:
one + two - three

tensor([[-0.0386, -0.0136, -0.0164, -0.0318,  0.0185, -0.0220,  0.0101,  0.0049,
          0.0098, -0.0225]], grad_fn=<SubBackward0>)